# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data=pd.read_csv("../output_data/weather_data.csv")
data.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Thompson,55.74,-97.86,59.00,100,75,4.14,CA,1594709574
1,Bagdarin,54.43,113.60,73.96,69,50,4.94,RU,1594709574
2,Umm Kaddadah,13.60,26.69,85.86,52,0,14.29,SD,1594709574
3,Bredasdorp,-34.53,20.04,46.40,93,100,16.11,ZA,1594709574
4,Micheweni,-4.97,39.83,80.11,70,0,11.56,TZ,1594709575


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
from config import gkey
gmaps.configure(api_key=gkey)

locations = data[["Lat", "Lng"]]
rating = data["Humidity"]
figure_layout = {
    'width': '1000px',
    'height': '750px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=data["Humidity"].max(), point_radius=2)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='750px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df=data.loc[(data["Max Temp"]>70)&(data["Max Temp"]<80)&(data["Cloudiness"]==0)&(data["Wind Speed"]<5)]
hotel_df.reset_index(inplace=True)
hotel_df.drop(columns=["index"],inplace=True)
hotel_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Escondido,15.85,-97.07,78.48,80,0,4.23,MX,1594709580
1,Cap Malheureux,-19.98,57.61,77.00,68,0,4.00,MU,1594709013
2,Acapulco de Juárez,16.86,-99.89,79.61,77,0,3.65,MX,1594709325
3,Rio Claro,-22.41,-47.56,71.01,55,0,1.99,BR,1594709594
4,Lázaro Cárdenas,17.96,-102.20,77.95,82,0,3.53,MX,1594709595
5,Tonj,7.27,28.68,78.76,76,0,4.23,SS,1594709602
6,Angoche,-16.23,39.91,75.47,59,0,3.40,MZ,1594709609
7,La Orilla,17.98,-102.23,77.25,84,0,3.80,MX,1594709614
8,Boa Vista,2.82,-60.67,77.00,88,0,2.24,BR,1594709336
9,Claremore,36.31,-95.62,78.01,80,0,1.99,US,1594709378


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels = []
#checker variable

# for loop to search through city coords
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# try/except to check if a city turns up 
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except (KeyError, IndexError):
        hotels.append("")

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Puerto Escondido,15.85,-97.07,78.48,80,0,4.23,MX,1594709580,Brisas de Zicatela
1,Cap Malheureux,-19.98,57.61,77.00,68,0,4.00,MU,1594709013,Grand Baie
2,Acapulco de Juárez,16.86,-99.89,79.61,77,0,3.65,MX,1594709325,Acapulco
3,Rio Claro,-22.41,-47.56,71.01,55,0,1.99,BR,1594709594,Rio Claro
4,Lázaro Cárdenas,17.96,-102.20,77.95,82,0,3.53,MX,1594709595,Lázaro Cárdenas
5,Tonj,7.27,28.68,78.76,76,0,4.23,SS,1594709602,Tonj
6,Angoche,-16.23,39.91,75.47,59,0,3.40,MZ,1594709609,Angoche Island
7,La Orilla,17.98,-102.23,77.25,84,0,3.80,MX,1594709614,Lázaro Cárdenas
8,Boa Vista,2.82,-60.67,77.00,88,0,2.24,BR,1594709336,Boa Vista
9,Claremore,36.31,-95.62,78.01,80,0,1.99,US,1594709378,Claremore


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

#Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='750px', margin='0 auto 0 auto', padding='1px', wi…